# İçindekiler
- [Rehber Hakkında](../../../../code/01.Introduce)
- [Kurulum](../../../../code/01.Introduce)
- [Sınırsız Üretim](../../../../code/01.Introduce)
- [Phi 3 İçin Konuşma](../../../../code/01.Introduce)
- [Regex](../../../../code/01.Introduce)
- [Seçim](../../../../code/01.Introduce)
- [Düşünce Zinciri](../../../../code/01.Introduce)
- [JSON Üretimi](../../../../code/01.Introduce)
- [HTML Üretimi](../../../../code/01.Introduce)


# Guidance Hakkında
Guidance, herhangi bir dil modelinin (LM) çıktısını kontrol etmek için kullanılan kanıtlanmış bir açık kaynaklı Python kütüphanesidir. Tek bir API çağrısıyla, modelin takip etmesi gereken kesin programatik kısıtlamaları (Python'da) ifade edebilir ve JSON, Python, HTML, SQL veya kullanım senaryosunun gerektirdiği herhangi bir yapıda yapılandırılmış çıktılar üretebilirsiniz.

Guidance, geleneksel yönlendirme tekniklerinden farklıdır. Kısıtlamaları, modelin çıkarım katmanında token token yönlendirilmesiyle uygular, daha yüksek kaliteli çıktılar üretir ve yüksek derecede yapılandırılmış senaryolarda maliyeti ve gecikmeyi %30–50 oranında azaltır.

Guidance hakkında daha fazla bilgi edinmek için [GitHub'daki halka açık depoyu](https://github.com/guidance-ai/guidance) ziyaret edebilir veya Microsoft Build'deki [Guidance Breakout Oturumunu](https://www.youtube.com/watch?v=qXMNPVVlCMs) izleyebilirsiniz.


# Kurulum
1. Guidance'i `pip install guidance --pre` komutuyla yükleyin
2. Azure'da bir Phi 3.5 mini uç noktası dağıtmak için https://ai.azure.com/explore/models/Phi-3.5-mini-instruct/version/2/registry/azureml adresine gidin ve "Deploy" düğmesine tıklayın
3. Uç noktanızın API anahtarını `AZURE_PHI3_KEY` adlı bir ortam değişkeninde ve URL'yi `AZURE_PHI3_URL` adlı bir ortam değişkeninde saklayın


In [ ]:
from guidance import gen, select, regex, user, assistant, system, json
from guidance.models import AzureGuidance
from json import loads as load_json_str
import os

phi3_url = os.getenv("AZURE_PHI3_URL")
phi3_api_key = os.getenv("AZURE_PHI3_KEY")
phi3_lm = AzureGuidance(f"{phi3_url}/guidance#auth={phi3_api_key}")

# Or, load from HuggingFace to run locally
# from guidance.models import Transformers
# phi3_lm = Transformers("microsoft/Phi-3-mini-4k-instruct")

# Kısıtlamasız üretim
Metin, `gen()` fonksiyonu kullanılarak herhangi bir kısıtlama olmadan üretilebilir. Bu, Guidance olmadan modeli kullanmakla aynıdır.

## Sohbet Formatlama
Birçok sohbet modeli gibi, Phi-3 kullanıcı ve asistan arasındaki mesajları belirli bir formatta bekler. Guidance, Phi-3'ün sohbet şablonunu destekler ve sohbet formatlamasını sizin için yönetir. Sohbet dönüşleri oluşturmak için, konuşmanın her bir bölümünü `with user()` veya `with assistant()` bloğuna yerleştirin. Sistem mesajını ayarlamak için bir `with system()` bloğu kullanılabilir.


In [22]:
lm = phi3_lm
with system():
    lm += "You are a helpful assistant. You have a cranky yet entertaining temperament."
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += gen(temperature=0.8, max_tokens=100)

## Token tasarrufu
Son derece yapılandırılmış senaryolarda, Guidance yalnızca gerekli olan tokenları üreterek tokenları atlayabilir, performansı artırabilir, verimliliği yükseltebilir ve API maliyetlerini düşürebilir. Üretilen tokenlar bu not defterinde vurgulanmış bir arka planla gösterilir. Zorunlu tokenlar ise vurgulanmadan gösterilir ve giriş tokenlarıyla aynı maliyete sahiptir, bu maliyet çıkış tokenlarının yaklaşık üçte biri olarak tahmin edilir.

*Not:* Kısıtlama olmadan yapılan ilk üretim örneği, herhangi bir tokenı zorlayamadı çünkü herhangi bir kısıtlama sağlamadık.


# Phi 3 İçin Konuşma  
Rehberlik ile, metni modelin yanıtlarına kolayca ekleyebilirsiniz. Bu, modelin çıktısını belirli bir yönde yönlendirmek istediğinizde faydalı olabilir.


In [5]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + gen(temperature=0.8, max_tokens=50)

# Regex ile sınırlandırma
Önceki örnekte, Phi 3 `Canberra` yanıtını verdikten sonra ek açıklamalarla devam etti. Modelin çıktısını tam olarak bir kelimeyle sınırlandırmak için bir regex kullanılabilir.


In [6]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + regex("[A-Z][a-z]+")

Regex ile yalnızca `Canberra` kelimesi oluşturulur.


# Birden Fazla Seçenek Arasından Seçim Yapma
Bazı olası seçenekler biliniyorsa, modelin bir seçenek listesinden seçim yapmasını sağlamak için `select()` fonksiyonunu kullanabilirsiniz.


In [23]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + select(["Washington", "Canberra", "Sydney", "Melbourne"])

`select()` ile yalnızca `Can` belirteci oluşturuldu. Çünkü yanıtı tamamlayabilecek tek seçenek `Canberra` olduğundan, kalan belirteçler zorlandı.


# Düşünce Zinciri
Düşünce zinciri, bir problemin adım adım işlenmesini teşvik ederek modelin çıktısının kalitesini artırmaya yardımcı olabilecek bir tekniktir. Genellikle, nihai bir sonuca ulaşmak için birden fazla istem turu gereklidir. Öncelikle, modele adım adım düşünmesini söyleyin. Ardından, modeli tekrar yönlendirerek nihai cevabı vermesini isteyin. Standart sohbet çıkarım API'leri ile bu işlem 2 API çağrısı gerektirir ve modelin oluşturduğu "düşünce zinciri" iki kez ücretlendirilir – bir kez model tarafından oluşturulduğunda çıktı jetonları olarak ve bir kez daha ikinci çağrı için giriş jetonları olarak. Guidance ile, tüm çok aşamalı süreç tek bir API çağrısının parçası olarak işlenir ve ücretlendirilir, böylece maliyet ve gecikme süresi azalır.


In [8]:
gsm8k_question = "Mark has a garden with flowers. He planted plants of three different colors in it. Ten of them are yellow, and there are 80% more of those in purple. There are only 25% as many green flowers as there are yellow and purple flowers. How many flowers does Mark have in his garden?"
lm = phi3_lm
with user():
    lm += gsm8k_question
with assistant():
    lm += "Let's think step by step. " + gen(temperature=0.8, max_tokens=500)
    # Prompt for the final answer, which should be a number. Store the output in an "answer" variable.
    lm += "\nTherefore, the final answer is: " + regex(r"\d+", name="answer")

print(f"Final answer: {lm['answer']}")

Final answer: 35


# JSON Oluşturma
Guidance, burada gösterilen kullanıcı profili şeması gibi bir JSON şeması veya pydantic modeliyle uyumlu JSON oluşturulmasını garanti etmek için kullanılabilir.


In [16]:
user_json_schema = load_json_str("""{
  "$schema": "http://json-schema.org/draft-07/schema#",
  "title": "User Profile",
  "type": "object",
  "properties": {
    "username": {
      "type": "string"
    },
    "age": {
      "type": "integer"
    },
    "email": {
      "type": "string"
    }
  },
  "additionalProperties": false
}
""")

lm = phi3_lm
with user():
    lm += "Generate a JSON object for a user profile. The profile should include a username, age, email, and nothing more."

with assistant():
    lm += json(schema=user_json_schema, temperature=1.0)

In [19]:
from pydantic import BaseModel

class UserProfile(BaseModel):
    username: str
    age: int
    email: str


lm = phi3_lm
with user():
    lm += "Generate a JSON object for a user profile. The profile should include a username, age, email, and nothing more."

with assistant():
    lm += json(schema=UserProfile, temperature=1.0)

## HTML Oluşturma

Guidance, kod oluşturmak ve programlama dilindeki sözdizimsel gereksinimlere uymak için de kullanılabilir. Bu bölümde, çok basit HTML web sayfaları yazmak için küçük bir Guidance programı oluşturacağız.

Web sayfasını daha küçük bölümlere ayıracağız ve her bir bölüm için kendi Guidance fonksiyonunu yazacağız. Daha sonra bu fonksiyonları birleştirerek bir HTML web sayfası oluşturacağız.
Son olarak, bu fonksiyonu Azure AI'de Guidance destekli bir model üzerinde çalıştıracağız.

*Not:* Bu, tam özellikli bir HTML oluşturucu olmayacak; amaç, bireysel ihtiyaçlarınız için yapılandırılmış çıktı oluşturmanın nasıl yapılabileceğini göstermektir.

Başlamak için Guidance'dan ihtiyaç duyduğumuz şeyleri içe aktarıyoruz:


In [ ]:
from guidance import guidance
from guidance.library import (
    zero_or_more,
    any_char_but,
    select,
    capture,
    with_temperature,
)
from guidance.models import Model

HTML web sayfaları oldukça yapılandırılmıştır ve sayfanın bu bölümlerini Guidance kullanarak 'zorlayacağız'.  
Modelden açıkça metin talep ettiğimizde, herhangi bir etiket olabilecek şeyleri içermediğinden emin olmamız gerekir - yani '<' ve '>' karakterlerini hariç tutmalıyız.


In [ ]:
@guidance(stateless=True)
def _gen_text(lm: Model):
    return lm + zero_or_more(any_char_but(["<", ">"]))

Ardından bu işlevi kullanarak rastgele bir HTML etiketinin içinde metin oluşturabiliriz:


In [ ]:
@guidance(stateless=True)
def _gen_text_in_tag(lm: Model, tag: str):
    lm += f"<{tag}>"
    lm += _gen_text()
    lm += f"</{tag}>"
    return lm

Sayfa başlığını oluşturalım. Bunun bir parçası olarak, bir sayfa başlığı oluşturmamız gerekiyor:


In [ ]:
@guidance(stateless=True)
def _gen_header(lm: Model):
    lm += "<head>\n"
    lm += _gen_text_in_tag("title") + "\n"
    lm += "</head>\n"
    return lm

HTML sayfasının gövdesi başlıklar ve paragraflarla doldurulacak.  
Her biri için bir fonksiyon tanımlayabiliriz:


In [ ]:
@guidance(stateless=True)
def _gen_heading(lm: Model):
    lm += select(
        options=[_gen_text_in_tag("h1"), _gen_text_in_tag("h2"), _gen_text_in_tag("h3")]
    )
    lm += "\n"
    return lm

@guidance(stateless=True)
def _gen_para(lm: Model):
    lm += _gen_text_in_tag("p")
    lm += "\n"
    return lm

Şimdi, HTML'nin gövdesini tanımlamak için fonksiyon.

Bu, birden fazla başlık ve paragraf oluşturmak için `select()` ile `recurse=True` kullanır:


In [ ]:
@guidance(stateless=True)
def _gen_body(lm: Model):
    lm += "<body>\n"
    lm += select(options=[_gen_heading(), _gen_para()], recurse=True)
    lm += "</body>\n"
    return lm

Sonraki adımda, tam HTML sayfasını oluşturan fonksiyona geliyoruz.  
HTML başlangıç etiketini ekliyoruz, ardından başlığı oluşturuyoruz, sonra gövdeyi oluşturuyoruz ve en son HTML bitiş etiketini ekliyoruz:


In [ ]:
@guidance(stateless=True)
def _gen_html(lm: Model):
    lm += "<html>\n"
    lm += _gen_header()
    lm += _gen_body()
    lm += "</html>\n"
    return lm

Kullanıcı dostu bir sarmalayıcı sağlıyoruz, bu da bize şunları yapma imkanı tanıyacak:
- Üretimin sıcaklığını ayarlamak
- Model nesnesinden oluşturulan sayfayı yakalamak


In [ ]:
@guidance(stateless=True)
def html(
    lm,
    name: str | None = None,
    *,
    temperature: float = 0.0,
):
    return lm + capture(
        with_temperature(_gen_html(), temperature=temperature),
        name=name,
    )

In [ ]:
lm = phi3_lm

lm += "Create a web page about your life story. Split your uplifting tale into multiple paragraphs with headings:\n"
lm += html(name="html_text", temperature=0.7)

Çıktıyı bir dosyaya yazabiliriz:


In [ ]:
with open('./sample_page.html', 'w') as html_file:
    html_file.write(lm["html_text"])

Ve [sonucu gör](../../../../code/01.Introduce/sample_page.html).



---

**Feragatname**:  
Bu belge, AI çeviri hizmeti [Co-op Translator](https://github.com/Azure/co-op-translator) kullanılarak çevrilmiştir. Doğruluk için çaba göstersek de, otomatik çevirilerin hata veya yanlışlık içerebileceğini lütfen unutmayın. Belgenin orijinal dili, yetkili kaynak olarak kabul edilmelidir. Kritik bilgiler için profesyonel insan çevirisi önerilir. Bu çevirinin kullanımından kaynaklanan yanlış anlamalar veya yanlış yorumlamalar için sorumluluk kabul etmiyoruz.
